In [1]:
# import socket and json
import socket
from time import sleep
import pickle

# Import required modules
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import time
import datetime

from textblob import TextBlob

In [2]:
def GetAllTextFromContentBody(contentBody):
    out_str = ''
    for x in contentBody:
        out_str += x.text
        
    return out_str

In [3]:
def GetDateOfArticle(publishedData):
    time_, GMT = publishedData.split('+')
    dt = datetime.datetime.strptime(time_, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
    return dt

In [4]:
def GetDateTimeOfArticle(soup):
    metatags = soup.find_all('meta',attrs={'name':'publish-date'})
    for tag in metatags:
        return GetDateOfArticle(tag['content'])

In [5]:
def GetSentimentalScore(text):
    seperator = '#'
    analysis = TextBlob(text)
    sentiment_value, confidence = analysis.sentiment
    print confidence
    if confidence*100 >= 70:                 
                if sentiment_value > 0.0 :
                    return (str(sentiment_value).encode('utf-8') +str(seperator).encode('utf-8') +str('positive').encode('utf-8') )
                    
                if sentiment_value < 0.0 :
                    return (str(sentiment_value).encode('utf-8') +str(seperator).encode('utf-8') +str('negetive').encode('utf-8') )
                    
                if sentiment_value == 0.0 :
                    return (str(sentiment_value).encode('utf-8') +str(seperator).encode('utf-8') +str('neutral').encode('utf-8') )
                    
    if confidence*100 < 70:
                    return (str(sentiment_value).encode('utf-8') +str(seperator).encode('utf-8') +str('invalid').encode('utf-8') )
                

                
    

In [6]:
def GetDataFromLinks(column):
        r = requests.get(x)
        html_content = r.text
        soup = BeautifulSoup(html_content , 'lxml')
        contentbody = soup.find_all("div", id=lambda value: value and value.startswith("content-body"))
        articleText = GetAllTextFromContentBody(contentbody)
        
        sentimentValue = GetSentimentalScore(articleText)
        articleDate = GetDateTimeOfArticle(soup)
        
        seperator = '#'
                
        #arr = ([str(articleDate.encode("utf-8"))],[str(articleText.encode("utf-8"))])
        #data_string = pickle.dumps(arr)
                    
        data_string =  str(articleDate).encode('utf-8')+ str(seperator).encode('utf-8') + str(sentimentValue).encode('utf-8') +'\n'

        return data_string    

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
host = "172.31.20.58"
port = 4212
s.bind((host,port))
s.listen(10)
        
        
while True:
    print('\nListening for a client at',host , port)
    conn, addr = s.accept()
    print('\nConnected by', addr)
    try:     
            
        df = pd.read_csv('all_links.csv')
        columnData = df['links']
        print (columnData.count)
        for x in columnData:            
            dataFromLink = GetDataFromLinks(x)
            print dataFromLink
            conn.send(dataFromLink)
            sleep(10)
        
       
    except socket.error:
        print ('Error Occured.\n\nClient disconnected.\n')
conn.close()